In [7]:
import time
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from datetime import datetime
from datetime import timedelta

import re
import pandas as pd

import numpy as np
import socket

import os
import csv, operator
import unidecode
from random import randint

In [8]:
cwd = os.getcwd()
read_date=(datetime.now() .strftime('%Y-%m-%d'))
read_time = time.strftime("%H:%M:%S")
date_file=((datetime.now()) .strftime('%Y%m%d%H%M'))
csvsalida = open(cwd+'/output/funda_purchase_'+date_file+'_out.csv', 'w', newline='')
salida = csv.writer(csvsalida, delimiter='|', quotechar='"', quoting=csv.QUOTE_ALL)
salida.writerow(['read_date', 'read_time','address','zip_code','price','living_surface',"parcel_surface",'rooms', "broker"])

105

In [9]:
cities = ["amsterdam", "rotterdam", "den-haag", "utrecht", "eindhoven"]
trx = ["koop", "huur"]

In [10]:
url ="https://www.funda.nl/koop/amsterdam/"
chrome_options = Options() 
driver = webdriver.Chrome(chrome_options=chrome_options) 
driver.get(url)
sleep(5)

In [11]:
for trx_type in trx:
    for city in cities:
        url ="https://www.funda.nl/"+trx_type+"/"+city+"/"
        try:
            driver.get(url)
        except:
            chrome_options = Options() 
            driver = webdriver.Chrome(chrome_options=chrome_options) 
            driver.get(url)
        sleep(5)
        try:
            accept_cookies = driver.find_element_by_xpath("//*[@id='onetrust-accept-btn-handler']")
            accept_cookies.click()
            sleep(2)
        except:
            pass
        
        keep_looping = 1
        first_page = 1
        while keep_looping == 1:
            soup=BeautifulSoup(driver.page_source,'lxml')
            listings = soup.findAll("div", class_= "search-result-content-inner")
            for i in range(0, len(listings)):
                try:
                    address = listings[i].find("h2", class_= "search-result__header-title fd-m-none").get_text().replace("\t", "").replace("\n", "").replace(",", "").strip()
                    zip_code = listings[i].find("h4", class_= "search-result__header-subtitle fd-m-none").get_text().replace("\t", "").replace("\n", "").replace(",", "").strip()
                    price = listings[i].find("span", class_= "search-result-price").get_text().replace("€ " ,"").replace("€" ,"").replace("." ,"").replace("," ,".")
                    living_surface = listings[i].find("span", title="Gebruiksoppervlakte wonen").get_text().replace("\t", "").replace("\n", "").replace(",", "").strip()
                    try:
                        parcel_surface = listings[i].find("span", title="Perceeloppervlakte").get_text().replace("\t", "").replace("\n", "").replace(",", "").strip()
                    except:
                        parcel_surface = "-"
                    rooms_container = listings[i].find("ul", class_= "search-result-kenmerken")
                    room_container = rooms_container.findAll("li")
                    rooms = room_container[1].get_text().replace("\t", "").replace("\n", "").replace(",", "").strip()
                    try:
                        broker = listings[i].find("span", class_="search-result-makelaar-name").get_text().replace("\t", "").replace("\n", "").replace(",", "").strip()
                    except:
                        broker = "-"
                    salida.writerow([read_date, read_time,address,zip_code,price,living_surface,parcel_surface,rooms, broker])
                except:
                    pass
            try:
                body_elem = driver.find_element_by_tag_name('body')
                body_elem.send_keys(Keys.END)
                for i in range(0,12): body_elem.send_keys(Keys.UP)
                sleep(0.5)
                for i in range(0,8): body_elem.send_keys(Keys.UP)  
                sleep(2)
                if first_page == 1:
                    next_page = driver.find_element_by_xpath("//*[@id='content']/form/div[2]/nav/a[1]")
                else:
                    next_page = driver.find_element_by_xpath("//*[@id='content']/form/div[2]/nav/a[2]")
                next_page.click()
                first_page += 1
                keep_looping = 1
                sleep(2)
            except:
                keep_looping = 0
                pass

del salida
csvsalida.close()